In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

# Reading Data

In [2]:
con = sqlite3.connect('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\Dataset\\database.sqlite') 

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 

In [3]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

In [4]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


# Data Cleaning: Deduplication

In [5]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print(final.shape)

#Checking to see how much % of data still remains
(final.shape[0] / filtered_data.shape[0])*100

(364173, 10)


69.25890143662969

In [6]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

# Text Preprocessing.

1. Removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [8]:
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')

In [9]:
def cleanhtml(sent):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sent)
    return cleantext

In [10]:
def cleanpunc(sent):
    cleaned = re.sub(r'[?|!|\|"|#]', r'', sent)
    cleaned = re.sub(r"[']", r'', cleaned)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned)
    return cleaned

In [21]:
i = 0
str1 = ' '
final_string = []
all_positive_words = [] # store words from positive reviews
all_negative_words = [] # words from negative reviews
s = ''

from tqdm import tqdm

for sent in tqdm(final['Text'].values):
    filtered_sent = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words) > 2)):
                if(cleaned_words.lower() not in stop):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sent.append(s)
                    if final['Score'].iloc[i] == 0:
                        all_positive_words.append(s)
                    if final['Score'].iloc[i] == 1:
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
                
    str1 = b" ".join(filtered_sent)
    
    final_string.append(str1)
    i = i + 1

100%|█████████████████████████████████████████████████████████████████████████| 364171/364171 [14:42<00:00, 412.43it/s]


In [22]:
final['CleanedText'] = final_string

In [23]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...


In [24]:
final['CleanedText'].iloc[0]

b'witti littl book make son laugh loud recit car drive along alway sing refrain hes learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg'

In [25]:
# this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and
# he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new 
# words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my
# son will STILL be able to recite from memory when he is  in college

# Featurization

## BAG OF WORDS

In [29]:
#BoW
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(final_string)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(final_string)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some feature names  ['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaargh', 'aaaaaaaaagghh', 'aaaaaaah', 'aaaaaaahhhhhh']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (364171, 71624)
the number of unique words  71624


## Bi-Grams and n-Grams.

In [30]:
# CountVectorizer documentation http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# you can choose these numebrs min_df=10, max_features=5000, of your choice
count_vect = CountVectorizer(ngram_range = (1,2), min_df = 10)#, max_features = 5000)
final_bigram_counts = count_vect.fit_transform(final_string)
print("the type of count vectorizer ",type(final_bigram_counts))
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (364171, 206995)
the number of unique words including both unigrams and bigrams  206995


## TF-IDF

In [31]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(final_string)
print("some sample features(unique words in the corpus)",tf_idf_vect.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf_vect.transform(final_string)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['aaa', 'aaah', 'aafco', 'aah', 'ab', 'aback', 'abandon', 'abbey', 'abbi', 'abbott']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (364171, 206995)
the number of unique words including both unigrams and bigrams  206995


## Word2Vec

In [27]:
# Train your own Word2Vec model using your own text corpus
i = 0
list_of_sentance=[]
for sentance in tqdm(final_string):
    list_of_sentance.append(sentance.split())

100%|██████████████████████████████████████████████████████████████████████| 364171/364171 [00:02<00:00, 126993.93it/s]


In [28]:
print(list_of_sentance[0])

[b'witti', b'littl', b'book', b'make', b'son', b'laugh', b'loud', b'recit', b'car', b'drive', b'along', b'alway', b'sing', b'refrain', b'hes', b'learn', b'whale', b'india', b'droop', b'love', b'new', b'word', b'book', b'introduc', b'silli', b'classic', b'book', b'will', b'bet', b'son', b'still', b'abl', b'recit', b'memori', b'colleg']


In [29]:
list_of_sentance[0][0]

b'witti'

In [30]:
# str(b'hello', encoding)
encoding = 'utf-8'
a = str(list_of_sentance[0][0], encoding)

In [31]:
encoding = 'utf-8'
list_to_sent_str = []
for sent in tqdm(list_of_sentance):
    s = []
    for w in sent:
        temp = str(w, encoding)
        s.append(temp)
    list_to_sent_str.append(s)

100%|███████████████████████████████████████████████████████████████████████| 364171/364171 [00:07<00:00, 47122.97it/s]


In [32]:
want_to_train_w2v = True

if want_to_train_w2v:
    # min_count = 2 considers only words that occured atleast 1 times
    # size = 50 dimension of the vector
    # workers = 4 number of computer used for computing
    
    w2v_model = Word2Vec(list_to_sent_str, min_count=2, size=50, workers=4)
    
    print(w2v_model.wv.most_similar('great'))
    
    print('='*50)
    
    print(w2v_model.wv.most_similar('worst'))

[('fantast', 0.8554610013961792), ('good', 0.8539867401123047), ('terrif', 0.8416933417320251), ('excel', 0.8376325368881226), ('awesom', 0.817306637763977), ('wonder', 0.7849027514457703), ('nice', 0.7649551630020142), ('perfect', 0.7407717108726501), ('fabul', 0.701228678226471), ('decent', 0.6834162473678589)]
[('nastiest', 0.8511088490486145), ('greatest', 0.7485708594322205), ('disgust', 0.7450594305992126), ('best', 0.7162209153175354), ('horribl', 0.7159982919692993), ('nicest', 0.7000775337219238), ('tastiest', 0.6878936290740967), ('terribl', 0.6872791051864624), ('wors', 0.6744000911712646), ('aw', 0.6682536005973816)]


In [33]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 2 times ",len(w2v_words)) # as min_count = 2
print("*"*50)
print("sample words ", w2v_words[0:50])

number of words that occured minimum 2 times  37669
**************************************************
sample words  ['witti', 'littl', 'book', 'make', 'son', 'laugh', 'loud', 'recit', 'car', 'drive', 'along', 'alway', 'sing', 'refrain', 'hes', 'learn', 'whale', 'india', 'droop', 'love', 'new', 'word', 'introduc', 'silli', 'classic', 'will', 'bet', 'still', 'abl', 'memori', 'colleg', 'grew', 'read', 'sendak', 'watch', 'realli', 'rosi', 'movi', 'incorpor', 'howev', 'miss', 'hard', 'cover', 'version', 'paperback', 'seem', 'kind', 'flimsi', 'take', 'two']


## Converting text into vectors using wAvg W2V, TFIDF-W2V

### Avg W2v

In [36]:
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_to_sent_str): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, 
                            # you might need to change this to 300 if you use google's w2v
    cnt_words = 0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████| 364171/364171 [12:56<00:00, 468.96it/s]

364171
50


### TFIDF weighted W2v

In [101]:
model = TfidfVectorizer(min_df = 2)
model.fit(final_string)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [102]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

In [103]:
len(tfidf_feat), len(w2v_words)

(35806, 37669)

In [104]:
tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_to_sent_str): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            
            tf_idf = dictionary[word]*(sent.count(word)/len(sent)) # this line evaluates idf * tf
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|████████████████████████████████████████████████████████████████████████| 364171/364171 [2:29:15<00:00, 40.66it/s]


In [105]:
tfidf_sent_vectors = np.asarray(tfidf_sent_vectors)
tfidf_sent_vectors_df = pd.DataFrame(tfidf_sent_vectors)

In [106]:
tfidf_sent_vectors_df.to_csv('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_tfidf.csv')